In [1]:
import pprint, json, math, os, sys, camelot
import fitz, pdfplumber
import pandas as pd
import numpy as np
import ocrmypdf
from collections import defaultdict

# dir_path = "C:\\Users\\Kaustubh.keny\\OneDrive - Cogencis Information Services Ltd\\Documents\\mywork-repo\\"
# fund_path = "C:\\Users\\Kaustubh.keny\\OneDrive - Cogencis Information Services Ltd\\Documents\\Jan 25\\"

dir_path = "C:\\Users\\rando\\OneDrive\\Documents\\mywork-repo"
fund_path =  "C:\\Users\\rando\\OneDrive\\Documents\\Jan 25"
sys.path.append(os.path.abspath(dir_path))

from app.utils import Helper
from app.parse_regex import *

dry_path = r'\data\output\DryRun.pdf'
fin_path = r'\data\input\financial_indices.xlsx'
mutual_fund = Helper.get_fund_paths(fund_path)

In [3]:
def extract_clipped_data(input:str, pages:list, bboxes:list):
        
        document = fitz.open(input)
        final_list = []
    
        for pgn in pages:
            page = document[pgn]
            
            all_blocks = [] #store every data from bboxes
            
            for bbox in bboxes:
                blocks, seen_blocks = [], set()  #store unique blocks based on content and bbox
                
                page_blocks = page.get_text('dict', clip=bbox)['blocks']
                for block in page_blocks:
                    if block['type'] == 0 and 'lines' in block: #type 0 means text block
                        #hash_key
                        block_key = (tuple(block['bbox']), tuple(tuple(line['spans'][0]['text'] for line in block['lines'])))
                        if block_key not in seen_blocks:
                            seen_blocks.add(block_key)
                            blocks.append(block)

                sorted_blocks = sorted(blocks, key=lambda x: (x['bbox'][1], x['bbox'][0]))
                all_blocks.append(sorted_blocks)

            final_list.append({
                "pgn": pgn,
                "block": all_blocks #will be list[list,list,..]
            })

        document.close()
        return final_list
    
def extract_data_relative_line(path: str, line_x: float, side: str):
    doc = fitz.open(path)
    pages = doc.page_count

    final_list = []

    for pgn in range(pages):
        page = doc[pgn]

        blocks = page.get_text("dict")["blocks"]
        sorted_blocks = sorted(blocks, key=lambda x: (x["bbox"][1], x["bbox"][0]))
        extracted_blocks = []

        # Keep track of blocks to avoid duplicates
        added_blocks = set()

        for block in sorted_blocks:
            block_id = id(block)  # Unique identifier for the block

            for line in block.get("lines", []):
                for span in line.get("spans", []):
                    origin = span["origin"]
                    x0, _ = origin

                    # Check the side condition
                    if side == "left" and x0 < line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added
                    elif side == "right" and x0 > line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added

      
        final_list.append({
            "pgn": pgn,
            "blocks": extracted_blocks
        })

    doc.close()

    return final_list
  
def get_clipped_data(input:str, bboxes:list[set], *args):
    
        document = fitz.open(input)
        final_list = []
        if args:
            pages = list(args)
        else:
            pages = [i for i in document.page_count]
        
        for pgn in pages:
            page = document[pgn]

            blocks = []
            for bbox in bboxes:
                blocks.extend(page.get_text('dict', clip = bbox)['blocks']) #get all blocks
            
            filtered_blocks = [block for block in blocks if block['type']== 0 and 'lines' in block]
            # sorted_blocks = sorted(filtered_blocks, key= lambda x: (x['bbox'][1], x['bbox'][0]))
            
             # Extract text from sorted blocks
            extracted_text = []
            for block in filtered_blocks:
                block_text = []
                for line in block['lines']:
                    line_text = " ".join(span['text'] for span in line['spans'])
                    block_text.append(line_text)
                extracted_text.append("\n".join(block_text))
            
            final_list.append({
            "pgn": pgn,
            "block": filtered_blocks,
            "text": extracted_text
            })
            
            
        document.close()
        return final_list
    
def get_clipped_text(input:str, bboxes:list[set],*args):

    document = fitz.open(input)
    final_list = []
    
    if args:
        pages = list(args)
    else:
        pages = [i for i in document.page_count]
    
    for pgn in pages:
        page = document[pgn]
        blocks = []
        for bbox in bboxes:
            blocks = page.get_text('text', clip = bbox).split('\n') #get all blocks
        final_list.append({
        "pgn": pgn,
        "block": blocks
        })   
    document.close()
    return final_list

def get_proper_fund_names(path: str, pages: list):
    doc = fitz.open(path)
    title = {}

    for pgn in pages:
        page = doc[pgn]
        blocks = page.get_text("dict")['blocks']
        text_all = " ".join(
            span["text"].strip()
            for block in blocks[:4]
            for line in block.get("lines", [])
            for span in line.get("spans", [])
            if span["text"].strip()
        )

        text_all = re.sub(r'[^A-Za-z0-9\s]+', '', text_all).strip()
        matches = re.findall(r"((?:LIC\s*MF|BLNCED|LOW)\s+.*?(?:FUND|ETF|FTF|FOF|PLAN|SAVER))", text_all, re.IGNORECASE)

        title[pgn] = matches[0] if matches else ""

    return title

In [30]:
sample_path  = mutual_fund["Navi Mutual Fund"]
sample_path

'C:\\Users\\rando\\OneDrive\\Documents\\Jan 25\\Navi Mutual Fund\\56_31-Jan-2025_1_FS.pdf'

In [33]:
lines = [
    ((165, 0), (165, 812)),# Vertical line
    ((0, 500), (812, 500))
]
pages = [12, 14,16]
bboxes = [(0, 0, 500, 200)]
pages = [i for i in range(1,110)]
Helper.draw_lines_on_pdf(sample_path, lines, bboxes, pages, dir_path +dry_path)


Modified PDF saved to: C:\Users\rando\OneDrive\Documents\mywork-repo\data\output\DryRun.pdf


In [70]:
def get_proper_fund_names(path: str):
    pattern = "(Navi.*?(?:fund|fund of fund))"
    title = {}
    
    with fitz.open(path) as doc:
        for pgn, page in enumerate(doc):
            text = " ".join(page.get_text("text", clip=(0, 0, 500, 200)).split("\n"))
            text = re.sub("[^A-Za-z0-9\\s\\-\\(\\).,]+", "", text).strip()
            if matches := re.findall(pattern, text,re.IGNORECASE):
                title[pgn] = matches[0]
    return title


In [71]:
title = get_proper_fund_names(sample_path)

In [72]:
title

{2: 'Navi Niy 50 Index Fund',
 4: 'Navi US Total Stock Market Fund',
 5: 'Navi NASDAQ 100  Fund',
 6: 'Navi Niy Next 50  Index Fund',
 8: 'Navi Niy Bank  Index Fund',
 10: 'Navi Niy Midcap 150  Index Fund',
 12: 'Navi Niy Midcap 150 In An open-ended equity scheme re Ni Period Navi Niy Midcap 150 Index fund',
 13: 'Navi Niy India  Manufacturing Index Fund',
 15: 'Navi Nifty India Manu In An open ended scheme replicating  tracking Nifty India  Period Navi Niy India Manufacturing Index Fund',
 16: 'Navi ELSS Tax Saver  Niy 50 Index Fund',
 18: 'Navi BSE Sensex  Index Fund',
 20: 'Navi Niy IT Index Fund',
 21: 'Navi Nifty IT Inde Period  Niy IT  TRI  Niy IT  TRI Navi Niy IT Index Fund',
 22: 'Navi Niy 500 Multicap 5025 Index Fund',
 29: 'Navi Niy Next 50 Index Fund',
 30: 'NAVI MUTUAL FUND'}

In [ ]:
# def get_proper_fund_names(self,path:str):
#         pattern = "([A-Z0-9\\s\\-]+\\s*PGIM INDIA)"
#         title = {}
#         with fitz.open(path) as doc:
#             for pgn,page in enumerate(doc):
#                 text = " ".join(page.get_text("text", clip = (0, 0, 260, 80)).split("\n"))
#                 text = re.sub(self.REGEX["escape"],"",text).strip()
#                 if matches:= re.findall(pattern,text):
#                     title[pgn] = f"PGIM INDIA {" ".join(matches[0][:-11].split())}".lower()
        
#         return title

In [90]:
data = Helper.get_all_pdf_data(sample_path)

In [91]:
data[16]

{'pgn': 16,
 'blocks': [{'number': 0,
   'type': 0,
   'bbox': (21.0, 29.66404914855957, 242.56117248535156, 51.64204788208008),
   'lines': [{'spans': [{'size': 18.0,
       'flags': 16,
       'font': 'Calibri,Bold',
       'color': -16777216,
       'ascender': 0.9520000219345093,
       'descender': -0.26899999380111694,
       'text': 'quant Active Fund ',
       'origin': (21.0, 46.800048828125),
       'bbox': (21.0,
        29.66404914855957,
        161.2740020751953,
        51.64204788208008)},
      {'size': 18.0,
       'flags': 0,
       'font': 'Calibri',
       'color': -16777216,
       'ascender': 0.9520000219345093,
       'descender': -0.26899999380111694,
       'text': ' ',
       'origin': (161.10000610351562, 46.800048828125),
       'bbox': (161.10000610351562,
        29.66404914855957,
        165.16799926757812,
        51.64204788208008)},
      {'size': 9.850000381469727,
       'flags': 0,
       'font': 'Calibri',
       'color': -16777216,
       'ascen

In [ ]:
import fitz
import ocrmypdf
import re

replacement_dict = {
    "LIC MF Large Cap Fund": r"LIC MF LARGE CAP FUND",
    "LIC MF Large & Mid Cap Fund": r"LIC MF LARGE[^\S\r\n]*&[^\S\r\n]*MID[^\S\r\n]*CAP[^\S\r\n]*FUND",
    "LIC MF Multi Cap Fund": r"LIC MF MULTICAP FUND",
    "LIC MF Value Cap Fund": r"LIC MF V[^\S\r\n]*'?T[^\S\r\n]*CAP[^\S\r\n]*FUND",
    "LIC MF Small Cap Fund": r"SMALLCAP FUND",
    "LIC MF Dividend Yield Fund": r"LIC MF DIV[^\S\r\n]*DEND[^\S\r\n]*YIELD[^\S\r\n]*FUND",
    "LIC MF Value Fund": r"LIC MF VALUE FUND",
    "LIC MF Focused Fund": r"LIC MF FOCUSED FUND",
    "LIC MF Infrastructure Fund": r"LIC MF INFRASTRUCTURE FUND",
    "LIC MF Point Manufacturing Fund": r"LIC MF Poin[^\S\r\n]*MANGFACTURING[^\S\r\n]*FUND",
    "LIC MF Banking & Financial Services Fund": r"BANKING[^\S\r\n]*&[^\S\r\n]*FINANC[^\S\r\n]*IAL[^\S\r\n]*SERVICES[^\S\r\n]*FUND",
    "LIC MF Healthcare Fund": r"HEALTHCARE FUND",
    "LIC MF ELSS Tax Saver Fund": r"LiC MF EL[^\S\r\n]*SS[^\S\r\n]*TAX[^\S\r\n]*SAVER?",
    "LIC MF Aggressive Hybrid Fund": r"LIC MF AGGRESSIVE HYBR[^\S\r\n]*D[^\S\r\n]*FUND",
    "LIC MF Balanced Advantage Fund": r"LIC MF B[^\S\r\n]*'?L[^\S\r\n]*\*[^\S\r\n]*NCED[^\S\r\n]*ADVANTAGE[^\S\r\n]*FUND",
    "LIC MF Equity Savings Fund": r"LIC MF EQUITY SAVINGS FUND",
    "LIC MF Conservative Hybrid Fund": r"LIC MF CONSER[^\S\r\n]*VATIVE[^\S\r\n]*nip?!?\)?[^\S\r\n]*FUND",
    "LIC MF Arbitrage Fund": r"LIC MF ARBITRAGE FUND",
    "LIC MF Overnight Fund": r"LIC MF OVERNIGHT FUND",
    "LIC MF Liquid Fund": r"LIC MF LIQUID FUND",
    "LIC MF Ultra Short Duration Fund": r"LIC MF ULTRA SHORT DURATION FUND",
    "LIC MF Low Duration Fund": r"LIC MF LOW[^\S\r\n]*'?DURATION[^\S\r\n]*FUND",
    "LIC MF Medium to Long Duration Fund": r"LIC MF MEDIUM[^\S\r\n]*[Ff]2[^\S\r\n]*LONG[^\S\r\n]*DURATION[^\S\r\n]*FUND",
    "LIC MF Banking & PSU Fund": r"LIC MF BANK[^\S\r\n]*&[^\S\r\n]*ONG[^\S\r\n]*&[^\S\r\n]*PSU[^\S\r\n]*FUND",
    "LIC MF Short Duration Fund": r"LIC MF[^\S\r\n]*SHORT[^\S\r\n]*DURATION[^\S\r\n]*FUND",
    "LIC MF Gilt Fund": r"LIC MGI FUND",
    "LIC MF Children's Fund": r"LIC MF[^\S\r\n]*l[^\S\r\n]*HILDRENS[^\S\r\n]*FUND",
    "LIC MF BSE Sensex ETF": r"BSE SENSEX ETF",
    "LIC MF Nifty 50 ETF": r"LIC MF NIFTY 50 ETF",
    "LIC MF Nifty 100 ETF": r"LIC MF NIFTY 100 ETF",
    "LIC MF Nifty Midcap 100 ETF": r"LIC MF NIFTY MIDCAP 100 ETF",
    "LIC MF Nifty 8-13 Yr G-Sec ETF": r"LIC MF NIFTY 8[^\S\r\n]*13[^\S\r\n]*YR[^\S\r\n]*G[^\S\r\n]*SECETF",
    "LIC MF BSE Sensex Index Fund": r"LIC MF BSE SENSEX INDEX FUND",
    "LIC MF Nifty 50 Index Fund": r"LIC MF NIFTY 50 INDEX FUND",
    "LIC MF Nifty Next 50 Index Fund": r"LIC MF NIFTY NEXT 50 INDEX FUND",
    "LIC MF Gold Exchange Traded Fund": r"LIC MF G[^\S\r\n]*2LD[^\S\r\n]*EXCHANGE[^\S\r\n]*TRADED[^\S\r\n]*FUND",
    "LIC MF Gold ETF Fund of Fund": r"LIC MF GSLD ETF",
}

def process_pdf_and_correct_funds(input_pdf, bbox = (0, 0, 400, 100)):
    clipped_pdf = input_pdf.replace(".pdf", "_clipped.pdf")
    ocr_pdf = input_pdf.replace(".pdf", "_ocr.pdf")
    
    # Clip the PDF
    with fitz.open(input_pdf) as doc:
        with fitz.open() as new_doc:
            for page_num in range(len(doc)):
                new_page = new_doc.new_page(width=bbox[2] - bbox[0], height=bbox[3] - bbox[1])
                new_page.show_pdf_page(new_page.rect, doc, page_num, clip=bbox)
            new_doc.save(clipped_pdf)
    
    ocrmypdf.ocr(clipped_pdf, ocr_pdf, deskew=True, force_ocr=True)
    
    pattern = r"((?:LI?i?C|BSE|BANK|SMALL|HEALTH).*?(?:FUND|Path|ETF|FTF|EOF|FOF|PLAN|SAVER|FUND\s*OF\s*FUND))"
    extracted_titles = {}

    with fitz.open(ocr_pdf) as doc:
        for page_num, page in enumerate(doc):
            page_content = page.get_text("text")
            text = " ".join(page_content.split("\n"))
            if matches := re.findall(pattern, text, re.IGNORECASE):
                extracted_titles[page_num] = matches[0]
            else:
                extracted_titles[page_num] = ""

    # corrected_funds = {} #correct_them
    # for idx, fund in extracted_titles.items():
    #     for correct_name, regex in replacement_dict.items():
    #         if re.search(regex, fund, re.IGNORECASE):
    #             corrected_funds[idx] = correct_name
    #             break

    return extracted_titles

corrected_titles = process_pdf_and_correct_funds(sample_path)


In [41]:
corrected_titles

{0: '',
 1: '',
 2: '',
 3: '',
 4: '',
 5: '',
 6: '',
 7: '',
 8: '',
 9: '',
 10: '',
 11: '',
 12: '',
 13: '',
 14: 'LIC MF LARGE CAP FUND',
 15: 'LIC MF LARGE& MID CAPFUND',
 16: '',
 17: 'LIC MF MULTICAP FUND',
 18: "LIC MF V't CAP FUND",
 19: 'SMALLCAP FUND',
 20: '',
 21: 'LIC MF DIV_DEND YIELD FUND',
 22: 'LIC MF VALUE FUND',
 23: 'LIC MF FOCUSED FUND',
 24: 'LIC MF INFRASTRUCTURE FUND',
 25: 'LIC MF Poin MANGFACTURING FUND',
 26: 'BANKING & FINANC-AL SERVICES FUND',
 27: 'HEALTHCARE FUND',
 28: 'LiC MF EL_SS TAX SAVER',
 29: 'LIC MF AGGRESSIVE HYBRiD FUND',
 30: 'LIC MF B’L*NCED ADVANTAGE FUND',
 31: 'LIC MF EQUITY SAVINGS FUND',
 32: 'LIC MF CONSERWATIVE nip?!) FUND',
 33: 'LIC MF ARBITRAGE FUND',
 34: '',
 35: '',
 36: 'LIC MF OVERNIGHT FUND',
 37: 'LIC MF LIQUID FUND',
 38: 'LIC MF ULTRA SHORT DURATION FUND',
 39: '',
 40: 'LIC MF LOW’ DURATION FUND',
 41: 'LIC MF MEDIUM-F2°LONG DURATION FUND',
 42: 'LIC MF BANK&ONG & PSU FUND',
 43: 'LIC MF_. SHORT DURATION FUND',
 44: '

In [8]:
dataf = get_proper_fund_names(sample_path,list(range(84)))

In [26]:
import camelot
def get_something(path: str):
    pattern = r"^(REDEMPTION PROCEEDS|FEATURES|ASSET ALLOCATION|FUND MANAGER|SCHEME|Sr\. No\.)$"

    with fitz.open(path) as doc:
        exists = defaultdict(int)
        for pgn, page in enumerate(doc):
            page_text = [t.strip() for t in page.get_text().split("\n")]
            for text in page_text:
                if re.match(pattern,text):
                    exists[pgn]+=1
                    
    
    return [str(pgn+1) for pgn, count in exists.items() if count > 4] #camelot starts pages from 1

pages = get_something(sample_path)

imp_pages = ",".join(pages)                

In [ ]:
tables = camelot.read_pdf(sample_path,pages=imp_pages, flavor="stream", table_areas= ["30,50,612,812"],column_tol = 4, split_text = True) #["30,50,612,812"]   

In [ ]:
import camelot
import pandas as pd
import numpy as np

def clean_column_name(x):
    cleaned = re.sub(r"[^A-Za-z0-9\s]", "", str(x), flags=re.IGNORECASE) #Corrected re.sub.
    cleaned = "_".join(cleaned.split())
    return cleaned

with pd.ExcelWriter("cleaned_tables.xlsx", engine="xlsxwriter") as writer:
    for i, table in enumerate(tables):
        df = table.df
        df = df.map(lambda x: " ".join(str(x).split("\n")).strip())
        df.replace("", np.nan, inplace=True)
        df.iloc[2:, 0] = df.iloc[2:, 0].ffill()
        df = df.iloc[1:, :]
        df.columns = df.iloc[0, :].apply(clean_column_name)
        # print(df.columns)
        df = df.iloc[1:, :]
        sheet_name = f"Table_{i+1}"

        # workbook = writer.book
        # worksheet = writer.sheets[sheet_name]

        # row = 1
        # last_value = None
        # start_row = None

        # for j, value in enumerate(df.iloc[:, 0], start=1):
        #     if pd.notna(value):  # New group found
        #         if last_value is not None and start_row is not None:
        #             worksheet.merge_range(start_row, 0, row - 1, 0, last_value)  # Merge previous block
        #         last_value = value
        #         start_row = row
        #     row += 1

        # # Merge last group
        # if last_value is not None and start_row is not None:
        #     worksheet.merge_range(start_row, 0, row - 1, 0, last_value)
    
        

print("Cleaned tables saved with merged spanning rows!")


NIPPON DATA

In [ ]:
def via_block(path:str):
    pattern = r"FUNDS AT A GLANCE"
    amc_pattern = "^(Nippon India|CPSE).*(?=Plan|Next 50|Sensex|Fund|Path|ETF|FOF|EOF|Funds|$)"
    imp_pages = []
    with fitz.open(path) as doc:
        for pgn, page in enumerate(doc):
                page_blocks = page.get_text("dict")["blocks"]
                sorted_blocks = sorted(page_blocks, key=lambda x: (x["bbox"][1], x["bbox"][0]))
                for block_count, block in enumerate(sorted_blocks[:10]):
                    if "lines" not in block:
                        continue
                    for line in block["lines"]:
                        for span in line["spans"]:
                            text = span["text"].strip()
                            if re.match(pattern,text):
                                imp_pages.append(pgn)
                                
        amc_fund = defaultdict(list)
    
        for pgn in imp_pages:
            page = doc[pgn]
            page_blocks = page.get_text("dict")["blocks"]
            sorted_blocks = sorted(page_blocks, key=lambda x: (x["bbox"][1], x["bbox"][0]))
            for block_count, block in enumerate(sorted_blocks):
                if "lines" not in block:
                    continue
                for line in block["lines"]:
                    for span in line["spans"]:
                        text = span["text"].strip()
                        color = span['color']
                        if re.match(amc_pattern,text)and color == -1:
                            # matches = re.findall(amc_pattern,text)
                            amc_fund[pgn].append(text)
                            
    return imp_pages, dict(amc_fund)          

In [ ]:
pages,amc = via_block(sample_path)
pages = list(map(str,[x+1 for x in pages]))

In [ ]:
final_scheme = defaultdict(list)
for key, value in amc.items():
    # print(key)
    set1 = ['Scheme Name']+value[:4]
    set2 = ['Scheme Name']+value[4:]
    final_scheme[key+1].append(set1)
    final_scheme[key+1].append(set2)

final_scheme = dict(final_scheme)

In [ ]:
imp_pages = ",".join(pages)
tables = camelot.read_pdf(sample_path,pages=imp_pages, flavor="lattice", line_scale = 40)  #table_areas = ["0,0,580,690"]            

In [ ]:
import pandas as pd
import numpy as np

with pd.ExcelWriter("merged_tables.xlsx", engine="openpyxl") as writer:
    count = 0  # Toggle between 0 and 1
    
    for i, table in enumerate(tables):
        df = table.df
        if df.shape[1] < 3:
            continue

    
        df = df.map(lambda x: " ".join(x.split("\n")).strip())
        df = df.map(lambda x: np.nan if not x.strip() else x)
        df.set_index(df.columns[0], inplace=True)

        for check in ["Scheme Name", "Market Capitalization"]:
            if check in df.index:
                df.drop(check, inplace=True)
                
        df_cleaned = df[~df.index.isna()]
        df_cleaned = df_cleaned[df_cleaned.index != ""]
        df_cleaned = df_cleaned.reset_index()
        df_fill = df_cleaned.ffill(axis=1)


        sch_vals = final_scheme[table.page][count]
        count = 1 - count  # Toggle between 0 and 1

        if len(sch_vals) == 5:
            df_fill.loc[-1] = sch_vals 
            df_fill = df_fill.sort_index().reset_index(drop=True)

        # Write to a new sheet
        df_fill.to_excel(writer, sheet_name=f"Table_{i+1}", index=False)

print("All tables saved in separate sheets in 'merged_tables.xlsx' 🚀")
